In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [80]:
df_wellcome = pd.read_csv('../data/processed/wellcomeMap.csv')
df_wellcome.head()

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude
0,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'sto...",22.283245,114.155105
1,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'sto...",22.324410,114.160454
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'sto...",22.306436,114.171362
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'sto...",22.315173,114.164415
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'sto...",22.310417,114.225745


## BusStop

In [81]:
df_busStop = pd.read_csv('../data/processed/busStop.csv')
df_busStop.head()

,id,STOP_ID,X,Y,LAST_UPDATE_DATE_STR,geometry,longitude,latitude
0,GS_BUS_STOP_VIEW.134,134,836298,815477,10/24/2022,POINT (114.17716544074156 22.278157267915223),114.177165,22.278157
1,GS_BUS_STOP_VIEW.135,135,836643,815593,10/24/2022,POINT (114.18051304472341 22.279204780631602),114.180513,22.279205
2,GS_BUS_STOP_VIEW.136,136,837048,815708,10/24/2022,POINT (114.1844428939664 22.280243167292028),114.184443,22.280243
3,GS_BUS_STOP_VIEW.137,137,837583,815779,10/24/2022,POINT (114.1896341986785 22.280884016074378),114.189634,22.280884
4,GS_BUS_STOP_VIEW.139,139,837920,816521,10/24/2022,POINT (114.19290497970104 22.287584455024394),114.192905,22.287584


In [82]:
from haversine import haversine, Unit

# Define a function to count the number of bus stops within a 1km radius of each shop
def count_bus_stops_within_1km(row, radius = 1000):
    shop_loc = (row['latitude'], row['longitude'])
    num_stops = 0
    for index, stop in df_busStop.iterrows():
        stop_loc = (stop['latitude'], stop['longitude'])
        distance = haversine(shop_loc, stop_loc, unit=Unit.METERS)
        if distance <= radius:  # 1km = 1000 meters
            num_stops += 1
    return num_stops

# Apply the function to each row in the shops dataframe and store the results in a new column
df_wellcome['num_bus_stops_within_1km'] = df_wellcome.apply(count_bus_stops_within_1km, axis=1)


In [83]:
df_wellcome.head()

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude,num_bus_stops_within_1km
0,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'sto...",22.283245,114.155105,116
1,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'sto...",22.324410,114.160454,117
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'sto...",22.306436,114.171362,98
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'sto...",22.315173,114.164415,94
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'sto...",22.310417,114.225745,78


# Mtr Station

In [84]:
df_mtr = pd.read_csv('../data/processed/mtr.csv')
df_mtr

,name,description,longitude,latitude,altitude
0,Asia World-Expo,"<img src=""https://lh3.googleusercontent.com/um...",113.942304,22.321828,0.0
1,Airport (HKG),"<img src=""https://lh3.googleusercontent.com/um...",113.936548,22.315947,0.0
2,Tsing Yi,"<img src=""https://lh3.googleusercontent.com/um...",114.107459,22.358321,0.0
3,Kowloon,"<img src=""https://lh3.googleusercontent.com/um...",114.161500,22.304701,0.0
4,Hong Kong,"<img src=""https://lh3.googleusercontent.com/um...",114.158179,22.284872,0.0
...,...,...,...,...,...
93,Tai Shui Hang,"<img src=""https://lh3.googleusercontent.com/um...",114.222622,22.408241,0.0
94,Shek Mun,"<img src=""https://lh3.googleusercontent.com/um...",114.208653,22.387887,0.0
95,City One,"<img src=""https://lh3.googleusercontent.com/um...",114.203535,22.382956,0.0
96,Sha Tin Wai,"<img src=""https://lh3.googleusercontent.com/um...",114.194770,22.376944,0.0


In [85]:
# Define a function to count the number of bus stops within a 1km radius of each shop
def count_mtr_station_within_1km(row, radius = 1000):
    shop_loc = (row['latitude'], row['longitude'])
    num_mtr = 0
    for index, stop in df_mtr.iterrows():
        mtr_loc = (stop['latitude'], stop['longitude'])
        distance = haversine(shop_loc, mtr_loc, unit=Unit.METERS)
        if distance <= radius:  # 1km = 1000 meters
            num_mtr += 1
    return num_mtr

# Apply the function to each row in the shops dataframe and store the results in a new column
df_wellcome['num_mtr_station_within_1km'] = df_wellcome.apply(count_mtr_station_within_1km, axis=1)


In [86]:
df_wellcome

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude,num_bus_stops_within_1km,num_mtr_station_within_1km
0,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'sto...",22.283245,114.155105,116,5
1,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'sto...",22.324410,114.160454,117,6
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'sto...",22.306436,114.171362,98,3
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'sto...",22.315173,114.164415,94,5
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'sto...",22.310417,114.225745,78,3
...,...,...,...,...,...,...,...,...,...,...
118,Wellcome Superstore,"69 Broadway, Mei Foo Sun Chuen, Hong Kong",ChIJT_ciK6v4AzQRs67ZP901Mik,3.8,147,"['supermarket', 'grocery_or_supermarket', 'sto...",22.335935,114.140741,49,3
119,Wellcome,"Hong Kong, Tsing Yi, 長 青 村長 青 商業 中心 地下 T1-T5 號",ChIJl-Hd7M34AzQRxa6yYEDFlaY,3.1,24,"['supermarket', 'grocery_or_supermarket', 'sto...",22.345011,114.109215,25,0
120,Wellcome,"G/F, No, 26-30 Wo Yi Hop Rd, Kwai Chung, Hong ...",ChIJ-5B442j5AzQR1WY8EluYXEI,3.5,24,"['supermarket', 'grocery_or_supermarket', 'sto...",22.365888,114.136964,73,1
121,Wellcome Supermarket,"608號 Lai Chi Kok Rd, Cheung Sha Wan, Hong Kong",ChIJ11HrihcBBDQRdA9WzXizBeY,0.0,0,"['supermarket', 'grocery_or_supermarket', 'sto...",22.333536,114.154778,122,5


# competitor

In [87]:
df_competitor = pd.read_csv("../data/processed/competitor.csv")
df_competitor.head()

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude
0,express by PARKnSHOP,"Chek Lap Kok, Scenic Rd, 8號地面上層H-I室 Units H & ...",ChIJBWcewr7iAzQRtFniX2gwnjI,4.9,8,"['supermarket', 'grocery_or_supermarket', 'sto...",22.298215,113.934145
1,Wellcome - Yat Tung Superstore,"Shop No 110, 1/F Eaton Shopping Centre, Yat Tu...",ChIJibJ9J6HiAzQR6f60A8w4neE,3.6,214,"['supermarket', 'grocery_or_supermarket', 'sto...",22.281394,113.934796
2,Market Place,Tung Chung,ChIJc_HM8LTiAzQR2y2gEnZ7rec,4.1,85,"['supermarket', 'grocery_or_supermarket', 'sto...",22.294966,113.951054
3,Wellcome,"Ying Tung Rd, Tung Chung",ChIJY0vZg8_jAzQRdi4RNFDW0Io,3.8,31,"['supermarket', 'grocery_or_supermarket', 'sto...",22.296000,113.953457
4,Wellcome Supermarket,"Government Offices, Mui Wo, 2 Ngan Kwong Wan R...",ChIJeQk2SitWATQRkLuiM6FNDSM,3.9,140,"['supermarket', 'grocery_or_supermarket', 'sto...",22.264011,114.001509


In [88]:
# Define a function to count the number of bus stops within a 1km radius of each shop
def count_competitor_within_1km(row, radius = 1000):
    shop_loc = (row['latitude'], row['longitude'])
    shop_place_id = row['place_id']
    num_competitor = 0
    for index, competitor_shop in df_competitor.iterrows():
        competitor_loc = (competitor_shop['latitude'], competitor_shop['longitude'])
        competitor_place_id = competitor_shop['place_id']
        distance = haversine(shop_loc, competitor_loc, unit=Unit.METERS)
        if (distance <= radius) and (shop_place_id!=competitor_place_id):  # 1km = 1000 meters
            num_competitor += 1
    return num_competitor

# Apply the function to each row in the shops dataframe and store the results in a new column
df_wellcome['num_competitor_within_1km'] = df_wellcome.apply(count_competitor_within_1km, axis=1)


# District

In [89]:
df_district = pd.read_csv('../data/processed/districtAll.csv')
df_district

,district,latitude,longitude,Total,Average household sie,Median income
0,Central and Western District,22.28219,114.14486,87700.0,2.6,42000.0
1,Eastern,22.27722,114.22519,188400.0,2.8,31500.0
2,Islands District,22.26382,113.96038,67400.0,2.7,28800.0
3,Kowloon City District,22.32866,114.19121,146500.0,2.7,29700.0
4,Kwai Tsing,22.35288,114.10004,176600.0,2.7,23300.0
5,Kwon Tong,22.31326,114.22581,247400.0,2.7,22200.0
6,North,22.49471,114.13812,111300.0,2.7,23300.0
7,Sai Kung Area,22.38198,114.27017,166900.0,2.9,37200.0
8,Sha Tin,22.38715,114.19534,245800.0,2.8,27100.0
9,Sham Shui Po,22.32989,114.16250,164700.0,2.6,22000.0


In [90]:
import pandas as pd
from haversine import haversine, Unit


# Define a function to calculate the distance between two points using the Haversine formula
def calc_distance(lat1, lon1, lat2, lon2):
    return haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)

# Create a new column in the shops dataframe to store the district name
df_wellcome['district'] = np.nan

# Iterate through each shop and find the nearest district
for i, shop in df_wellcome.iterrows():
    min_distance = float('inf')
    district_name = ''
    for j, district in df_district.iterrows():
        distance = calc_distance(shop['latitude'], shop['longitude'], district['latitude'], district['longitude'])
        if distance < min_distance:
            min_distance = distance
            district_name = district['district']
    df_wellcome.at[i, 'district'] = district_name



In [91]:
df_wellcome

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude,num_bus_stops_within_1km,num_mtr_station_within_1km,num_competitor_within_1km,district
0,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'sto...",22.283245,114.155105,116,5,4,Central and Western District
1,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'sto...",22.324410,114.160454,117,6,14,Sham Shui Po
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'sto...",22.306436,114.171362,98,3,13,Yau Tsim Mong
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'sto...",22.315173,114.164415,94,5,15,Yau Tsim Mong
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'sto...",22.310417,114.225745,78,3,4,Kwon Tong
...,...,...,...,...,...,...,...,...,...,...,...,...
118,Wellcome Superstore,"69 Broadway, Mei Foo Sun Chuen, Hong Kong",ChIJT_ciK6v4AzQRs67ZP901Mik,3.8,147,"['supermarket', 'grocery_or_supermarket', 'sto...",22.335935,114.140741,49,3,5,Sham Shui Po
119,Wellcome,"Hong Kong, Tsing Yi, 長 青 村長 青 商業 中心 地下 T1-T5 號",ChIJl-Hd7M34AzQRxa6yYEDFlaY,3.1,24,"['supermarket', 'grocery_or_supermarket', 'sto...",22.345011,114.109215,25,0,2,Kwai Tsing
120,Wellcome,"G/F, No, 26-30 Wo Yi Hop Rd, Kwai Chung, Hong ...",ChIJ-5B442j5AzQR1WY8EluYXEI,3.5,24,"['supermarket', 'grocery_or_supermarket', 'sto...",22.365888,114.136964,73,1,3,Tsuen Wan District
121,Wellcome Supermarket,"608號 Lai Chi Kok Rd, Cheung Sha Wan, Hong Kong",ChIJ11HrihcBBDQRdA9WzXizBeY,0.0,0,"['supermarket', 'grocery_or_supermarket', 'sto...",22.333536,114.154778,122,5,14,Sham Shui Po


In [92]:
df_wellcome['types'].unique()

array(["['supermarket', 'grocery_or_supermarket', 'store', 'food', 'point_of_interest', 'establishment']",
       "['supermarket', 'grocery_or_supermarket', 'food', 'point_of_interest', 'store', 'establishment']",
       "['store', 'point_of_interest', 'establishment']"], dtype=object)

In [93]:


# create a map centered at Hong Kong
m = folium.Map(location=[22.3193, 114.1694], zoom_start=11)

# iterate through the rows of the dataframe and add markers to the map
for index, row in df_wellcome.iterrows():
    name = row['formatted_address']
    lat = row['latitude']
    lon = row['longitude']
    popup = folium.Popup(name)
    marker = folium.Marker(location=[lat, lon], popup=popup)
    marker.add_to(m)

# display the map
m


In [94]:
df_wellcome.to_csv("../data/processed/wellcomeAnalysis.csv",index=False)